# Projet DATA732

ETUDIANTS : RISS Ryan, PRUVOST Jordan

FILLIERE : IDU4

--------------------------------------

## Travail préliminaire (rien de sexuel je vous rassure)

On ajoute les librairies utilisées (notamment Plotly)

In [1]:
import json
import os
import pprint
import plotly.express as px
import pandas as pd

On crée le corpus. Il est composé de 4 .json pour les 4 sites (on prendra l'ensemble des articles) :

- Sputnik (média pro russe)
- PressTV (média pro iranien)
- FDeSouche (média très à droite)
- E&R (pas besoin de préciser "c'est écrit dans son livre" https://www.youtube.com/watch?v=9NmoQ1wohQc)

On crée une liste appellée "allDataCorpus" pour ne pas perdre du temps à parser les json à chaque fois.

In [8]:
corpusFiles = ["topaz-data732--all--www.egaliteetreconciliation.fr--20190101--20211231.json", 
"topaz-data732--all--fr.sputniknews.africa--20190101--20211231.json", 
"topaz-data732--all--french.presstv.ir--20190101--20211231.json",
"topaz-data732--all--www.fdesouche.com--20190101--20211231.json"]

allDataCorpus = []

for file in corpusFiles:
    # open and load file
    f = open(file, 'r', encoding='utf-8')
    data = json.loads(f.read())
    siteName = f.name.split(".")[1]
    allDataCorpus.append({"siteName" : siteName, "data": data})
    f.close()

## Analyse du nombre d'articles

On crée une fonction nous permettant de connaître le nombre d'articles par mois. On sépare les données selon le site.

On notera qu'on triera les données avec un Bubble Sort pour des soucis d'optimisation.

In [109]:
def getNumberArticlePerMonth(dataCorpus):
    numberArticlePerMonth = []
    for file in dataCorpus:
        data = file["data"]
        siteName = file["siteName"]
        years = data["metadata-all"]["fr"]["month"].keys()
        for year in years:
            for month in data["metadata-all"]["fr"]["month"][year].keys():
                num = data["metadata-all"]["fr"]["month"][year][month]["num"]
                numberArticlePerMonth.append({"yearMonth" : (month + "-" + year), "count" : num, "site": siteName})

    #Bubble Sort
    good = False

    while(not good):
        good = True
        for i in range(0, len(numberArticlePerMonth) - 1):
            date1 = numberArticlePerMonth[i]["yearMonth"]
            date2 = numberArticlePerMonth[i+1]["yearMonth"]
            (month1, annee1) = date1.split("-")
            (month2, annee2) = date2.split("-")
            if(int(annee1) > int(annee2)):
                numberArticlePerMonth[i], numberArticlePerMonth[i+1] = numberArticlePerMonth[i+1], numberArticlePerMonth[i]
                good = False
            if(annee1 == annee2 and int(month2) < int(month1)):
                numberArticlePerMonth[i], numberArticlePerMonth[i+1] = numberArticlePerMonth[i+1], numberArticlePerMonth[i]
                good = False

    # On crée le bar chart

    fig = px.bar(numberArticlePerMonth, x='yearMonth', y='count', color="site")
    fig.show()

In [110]:
getNumberArticlePerMonth(allDataCorpus)

On peut déjà remarquer qu'il y a une inégalité du nombre d'article dans le corpus selon les sites.

Par exemple le média Sputnik est bien plus représenté que les autres.

## Analyses sur les mots-clés

On crée une fonction qui nous renvoie les mots-clés et leurs nombres d'itérations pour un corpus donné.

On peut passer directement par ["metadata-all"]["fr"]["all"]["kws"] pour cette fonction.

In [111]:
def getKeywords(dataCorpus, limit):
    """
    dataCorpus (str array) : les fichiers parser du corpus avec le nom de site
    limit (int) : pour prendre les x mots les plus communs
    """
    keywords = {}
    for file in dataCorpus:
        data = file["data"]
        kws = data["metadata-all"]["fr"]["all"]["kws"]
        
        for kw,iters in kws.items():
            if(kw in keywords):
                keywords[kw] += iters
            else:
                keywords[kw] = iters
    #Sort
    keywords = sorted(keywords.items(), key=lambda x:x[1], reverse=True)
    
    # Mise en forme pour plotly
    commonWords = []
    for f,s in keywords:
        commonWords.append({"keyword": f, "iteration": s})

    # On prend seulement les x mots les plus communs (selon le param limit)
    commonWords = commonWords[0:limit]
    fig = px.bar(commonWords, x='keyword', y='iteration')
    fig.show()

On crée un bar chart des 100 mots-clés les plus communs du corpus.

In [15]:
getKeywords(allDataCorpus, 100)

On peut remarquer que le mot "russe" est le 11e mot-clé le plus commun. Ce qui n'est pas choquant étant donné que le media sputnik est surreprésenté.

### Analyses plus poussées sur les mots-clés

#### Voir les liens entre les mots-clés

On crée une fonction qui donne pour un corpus donné et un mot-clé donné les mots-clés relatifs au mot-clé choisi.

Ici on est obligé de faire plusieurs boucles for imbriquées car nous devons parcourir article par article.

In [126]:
def showLinkedKws(dataCorpus, pickedKw, limit):
    linkedKws = {}
    for file in dataCorpus:
        data = file["data"]
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        kws = article["kws"]
                        if(pickedKw in kws):
                            for kw,iters in kws.items():
                                if(kw != pickedKw):
                                    if(kw in linkedKws):
                                        linkedKws[kw] += 1
                                    else:
                                        linkedKws[kw] = 1
    
    #Sort
    linkedKws = sorted(linkedKws.items(), key=lambda x:x[1], reverse=True)

    # Mise en liste de dictionnaires pour plotly
    commonLinkedWords = []
    for f,s in linkedKws:
         commonLinkedWords.append({"keyword": f, "iteration": s})

    commonLinkedWords = commonLinkedWords[0:limit]

    # Bar chart
    fig = px.bar(commonLinkedWords, x='keyword', y='iteration', title= "Linked keywords : " + pickedKw)
    fig.show()

Un exemple avec le mot-clé "juif" ( mot totalement choisi au hasard :) )

In [127]:
pickedKeyword = "juif"

commonLinkedWords = showLinkedKws(allDataCorpus, pickedKeyword, 100)

Les termes de "Israël" ou bien de "guerre" sont par exemple présents.

On peut aussi voir les personnalités en lien avec un mot-clef :

In [124]:
def showLinkedPeople(dataCorpus, pickedKw, limit):
    linkedPeople = {}
    for file in dataCorpus:
        data = file["data"]
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        people = article["per"]
                        kws = article["kws"]

                        if(pickedKw in kws):
                            for p,iter in people.items():
                                if(p in linkedPeople):
                                    linkedPeople[p] += 1
                                else:
                                    linkedPeople[p] = 1
    #Sort
    linkedPeople = sorted(linkedPeople.items(), key=lambda x:x[1], reverse=True)


    # Mise en liste de dictionnaire pour plotly
    commonLinkedPeople = []
    for f,s in linkedPeople:
        commonLinkedPeople.append({"person": f, "iteration": s})

    commonLinkedPeople = commonLinkedPeople[0:limit]

    # Bar chart 
    
    fig = px.bar(commonLinkedPeople, x='person', y='iteration',title= "Linked people : " + pickedKw)
    fig.show()

In [125]:
pickedKeyword = "juif"

showLinkedPeople(allDataCorpus, pickedKeyword, 100)

### Test sur d'autres mot-clés

On peut faire une liste de mots-clés et faire des analyses sur notre corpus.

In [128]:
pickedKeywords = ["arabe", "attentat", "guerre", "complot"]

pickedLimit = 75

for pickedKw in pickedKeywords:
    showLinkedKws(allDataCorpus, pickedKw, pickedLimit)
    showLinkedPeople(allDataCorpus, pickedKw, pickedLimit)

Avec plusieurs keywords on a souvent les noms d'Alain Soral ou d'Eric Zemmour qui reviennent.

On peut ainsi essayer de savoir si cette corrélation vient du corpus en général ou d'un ou de médias en particulier.

On crée une nouvelle fonction qui nous donne aussi le média.

In [130]:
def showLinkedPeopleWithSite(dataCorpus, pickedKw, limit):
    linkedPeopleWithSite = []
    for file in dataCorpus:
        data = file["data"]
        siteName = file["siteName"]
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        people = article["per"]
                        kws = article["kws"]
                        if(pickedKw in kws):
                            for p,iter in people.items():
                                # Tester si la personne associé au site existe déjà
                                isNew = True
                                for e in linkedPeopleWithSite:
                                    if e["person"] == p:
                                        e["iteration"] += iter
                                        dicts = e["iterationWithSite"]
                                        if siteName in dicts:
                                            dicts[siteName] += iter
                                        else:
                                            dicts[siteName] = iter
                                        isNew = False
                                        break
                                if(isNew):
                                    linkedPeopleWithSite.append({"person": p, "iteration": iter, "iterationWithSite": {siteName: iter}})

    linkedPeopleWithSite = sorted(linkedPeopleWithSite, key=lambda x:x['iteration'], reverse=True)

    commonLinkedPeopleWithSite = linkedPeopleWithSite[0:limit]

    commonLinkedPeopleWithSiteSplitted = []
    for e in commonLinkedPeopleWithSite:
        person = e["person"]
        for site, iteration in e["iterationWithSite"].items():
           commonLinkedPeopleWithSiteSplitted.append({"person": person, "iteration": iteration, "site": site})

    df = pd.DataFrame(commonLinkedPeopleWithSiteSplitted)
    
    fig = px.bar(df, x='person', y='iteration', color='site',category_orders={'person': df.person } ,title= "Linked people with site : " + pickedKw)
   
    fig.show()

In [138]:
pickedKeywords = ["juif", "arabe", "attentat", "guerre", "complot"]

pickedLimit = 75

for pickedKw in pickedKeywords:
    showLinkedPeopleWithSite(allDataCorpus, pickedKw, pickedLimit)

Ainsi les noms d'Eric Zemmour et d'Alain Soral sont essentiellement présent sur les articles d'E&R.

## Aller plus loin

On peut ainsi faire la même chose avec les organisations, localisations, etc.

On peut ainsi regrouper les mots-clés, personnes, organisations et localisations comme des "sujets" (ou topics en anglais) en lien avec un autre sujet.

On crée une fonction permettant de faire ceci tout en nous donnant le site qui y correspond.

In [141]:
def showLinkedTopics(corpusData, pickedTopic, limit):
    linkedTopics = []
    for file in corpusData:
        data = file["data"]
        siteName = file["siteName"]
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:

                        #On prend tous les topics de l'article
                        topicsArticle = article["per"] | article["kws"] | article["loc"] | article["org"]
                        
                        
                        if(pickedTopic in topicsArticle):
                            del topicsArticle[pickedTopic] # On supprime bien sûr le topic choisi
                            for t,iter in topicsArticle.items():
                                # Tester si les topics associés son déjà dans la liste sinon on l'ajoute
                                isNew = True
                                for e in linkedTopics:
                                    if e["topic"] == t :
                                        e["iteration"] += iter
                                        dicts = e["iterationWithSite"]
                                        if siteName in dicts:
                                            dicts[siteName] += iter
                                        else:
                                            dicts[siteName] = iter
                                        isNew = False
                                        break
                                if(isNew):
                                    linkedTopics.append({"topic": t, "iteration": iter, "iterationWithSite": {siteName: iter}})
    # Sort
    linkedTopics = sorted(linkedTopics, key=lambda x:x['iteration'], reverse=True)

    commonLinkedTopics = linkedTopics[0:limit]

    # Split les entrées par site
    commonLinkedTopicsSplitted = []
    for e in commonLinkedTopics:
        topic = e["topic"]
        for site, iteration in e["iterationWithSite"].items():
           commonLinkedTopicsSplitted.append({"topic": topic, "iteration": iteration, "site": site})

    df = pd.DataFrame(commonLinkedTopicsSplitted)

    #print(df)

    # Bar chart
    
    fig = px.bar(df, x='topic', y='iteration', color='site',category_orders={'topic': df.topic } ,title= "Linked topics : " + pickedTopic)
    fig.show()


In [142]:
# Corpus de test si l'execution prend trop de temps avec le corpus contenant tous les articles

testCorpus = ['topaz-data732--mali--www.egaliteetreconciliation.fr--20190101--20211231.json', 
'topaz-data732--mali--www.fdesouche.com--20190101--20211231.json',
'topaz-data732--mali--fr.sputniknews.africa--20190101--20211231.json',
'topaz-data732--mali--french.presstv.ir--20190101--20211231.json']

allDataTestCorpus = []

for file in testCorpus:
    # open and load file
    f = open(file, 'r', encoding='utf-8')
    data = json.loads(f.read())
    siteName = f.name.split(".")[1]
    allDataTestCorpus.append({"siteName" : siteName, "data": data})
    f.close()

#########################################################


pickedKeywords = ["juif", "arabe", "attentat", "guerre", "complot"]

pickedLimit = 75

for pickedKw in pickedKeywords:
    showLinkedTopics(allDataCorpus, pickedKw, pickedLimit)

On remarque ainsi que selon le sujet abordé la répartition des médias n'est pas la même. En effet, par exemple pour les topics "Arabe" ou "Guerre" les topics liés font la plupart partis de pressTV. Ce qui n'est pas étonnant car pressTV est un média pro-irannien.

## Améliorations ?

On pourrait améliorer les scripts en supprimant les doublons. Par exemple France et france ou seul une majuscule change.

De plus, des sujets très similaires comme par exemple "israël" et "israélien" peuvent être regroupés en un seul sujet.

Ces améliorations pourront permettre d'analyser le corpus avec plus de rigueur.